Adapted from
https://colab.research.google.com/github/huggingface/deep-rl-class/blob/master/notebooks/unit4/unit4.ipynb#scrollTo=V8oadoJSWp7C

In [1]:

%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1
!pip install imageio
!pip install gym[atari]
!pip install autorom[accept-rom-license]

In [2]:
# # Virtual display
# from pyvirtualdisplay import Display

# virtual_display = Display(visible=0, size=(1400, 900))
# virtual_display.start()

In [3]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

  Cloning https://github.com/ntasfi/PyGame-Learning-Environment.git to /tmp/pip-req-build-cr05a2dm
  Running command git clone --filter=blob:none --quiet https://github.com/ntasfi/PyGame-Learning-Environment.git /tmp/pip-req-build-cr05a2dm
  Resolved https://github.com/ntasfi/PyGame-Learning-Environment.git to commit 3dbe79dc0c35559bb441b9359948aabf9bb3d331
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/simoninithomas/gym-games to /tmp/pip-req-build-ok3_fe0k
  Running command git clone --filter=blob:none --quiet https://github.com/simoninithomas/gym-games /tmp/pip-req-build-ok3_fe0k
  Resolved https://github.com/simoninithomas/gym-games to commit f31695e4ba028400628dc054ee8a436f28193f0b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing m

In [4]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gym
import gym_pygame

# Hugging Face Hub
# from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
import imageio

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
print(device)

cpu


In [7]:
env_id = "ALE/Pong-v5"
# Create the env
env = gym.make(env_id,obs_type="grayscale",full_action_space=False)

# Create the evaluation env
eval_env = gym.make(env_id,obs_type="grayscale",full_action_space=False)

# Get the state space and action space
s_size = gym.spaces.utils.flatten_space(env.observation_space).shape[0]
a_size = env.action_space.n

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
/home/codespace/.python/current/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (210, 160)
  logger.warn(


In [8]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

The State Space is:  33600
Sample observation [[ 39 197 216 ...  61   6 118]
 [210  70 181 ... 149 138  11]
 [ 52 235 129 ... 123 139 169]
 ...
 [132 218  46 ...  64  62  47]
 [160  74  82 ...  53 153 247]
 [121 136  20 ...  23  22 156]]


In [9]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  6
Action Space Sample 2


In [59]:
from os import listdir
from os.path import isfile, join

save_dir = "diffusion training data/"

# Store the state space and action space for the diffusion model to use later.
observations = []
actions = [] 
rewards = [] 
dones = [] # Store True if the frame ended the episode (terminate or truncate), False otherwise 

def add_transition(observation, action, reward, done):
    """
    Observations: What the agent sees (dimensions are 33600 x 1)
    Actions: What the agent does (dimensions are 6 x 1)
    Rewards: The reward the agent gets (range is -1 to 1)
    Dones: If the episode is over (True or False)
    """
    observations.append(observation)
    actions.append(action)
    rewards.append(reward)
    dones.append(done)

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
def convert_to_tensors(observations,actions,rewards,dones):
    #create lists to organize the training data.
    inputs = []
    truths = []

    h = observations[0].shape[0] #height of a pong image
    w = observations[0].shape[1] #width of a pong image

    #take 2 images as input, return one image (the model should need two input images to determine ball velocity)
    for i in range(len(observations)-2):
        #expand action to a matrix the size of a pong image, and add it as a seperate channel in a tensor image later
        act = np.broadcast_to(actions[i+1], (h,w) )
        rew = rewards[i+2]
        don = dones[i+2]


        input = np.stack((observations[i], observations[i+1], act), axis=0) #put the channels together so my life is easier and the nn pays a lot of attention to act
        truth = (observations[i+2],rew,don) #tuple of outputs
        inputs.append(input)
        truths.append(truth)


    inputs = np.stack(inputs,axis=0) # shape of (n,3,h,w)
    truths = (
        np.expand_dims(np.stack([t[0] for t in truths],axis = 0), axis=1),#(n,1,h,w)
        np.fromiter((t[1] for t in truths),dtype="float"),#(n)
        np.fromiter((t[2] for t in truths),dtype="float")#(n)
    )

    # print(inputs.shape) # (n,3,h,w)
    # print(truths[0].shape) # (n,1,h,w)
    # print(truths[1].shape) # (n,)
    # print(truths[2].shape) # (n,)

    inputs = torch.tensor(inputs,device=device,dtype=torch.float)
    truths = (
        torch.tensor(truths[0],device=device,dtype=torch.float),
        torch.tensor(truths[1],device=device,dtype=torch.float),
        torch.tensor(truths[2],device=device,dtype=torch.float)
    )
    return inputs,truths

def save_transitions():
    """
    divide each array into sets of 1000 and save it in chunks, so it can be loaded with a dataloader.
    """
    assert len(observations) == len(actions) == len(rewards) == len(dones)
    #change bool in dones to floats
    for i in range(len(dones)):
        dones[i] = float(dones[i])

    existing_files = [f for f in listdir(save_dir) if isfile(join(save_dir, f))]
    j = None
    if existing_files:
        #This grabs the largest integer out of all the filenames (filter the string for digit chars, convert those chars to an int)
        j = max([int(''.join([c for c in f if c.isdigit()])) for f in existing_files])
    else:
        j = -1
    j += 1

    n = 1000 #hardcoded.

    #iterate while chunking it. 
    for i,obs,act,rew,don in zip(range(j,len(observations)),chunks(observations,n),chunks(actions,n),chunks(rewards,n),chunks(dones,n)):
        if len(obs) < n: # discard if its not big enough for a chunk.
            break
        np.savez_compressed(file=save_dir + "transitions{i}.npz".format(i=i), observations = obs,actions=act,rewards=rew,dones=don)
        inputs,truths = convert_to_tensors(obs,act,rew,don)
        torch.save(inputs,save_dir+"inputs{i}.pt".format(i=i))
        torch.save(truths,save_dir+"truths{i}.pt".format(i=i))

    

# def load_transitions():
#     import os.path
#     if not os.path.isfile(save_dir + "observations.npz"):
#         print("No saved transitions found")
#         return()
#     #its stored as a dict so grab out the single array. and 0th axis should be the number of transitions
#     observations = np.load(save_dir + "observations.npz")["arr_0"]
#     actions = np.load(save_dir + "actions.npz")["arr_0"]
#     rewards = np.load(save_dir + "rewards.npz")["arr_0"]
#     dones = np.load(save_dir + "dones.npz")["arr_0"]
#     assert(len(observations) == len(actions) == len(rewards) == len(dones))
#     print("loaded transitions of length", len(observations))
    
    

#     return observations, actions, rewards, dones

In [60]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state.flatten()).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

In [61]:
debug_policy = Policy(s_size, a_size, 64).to(device)
debug_policy.act(env.reset()[0])

(5, tensor([-6.1989e-06], grad_fn=<SqueezeBackward1>))

In [62]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # load_transitions()
    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100)
    scores = []
    # Line 3 of pseudocode
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()[0]
        # Line 4 of pseudocode
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, terminated, truncated, info = env.step(action)
            rewards.append(reward)
            add_transition(state, action, reward, terminated or truncated)
            if terminated or truncated:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        # Line 6 of pseudocode: calculate the return
        returns = deque(maxlen=max_t) 
        n_steps = len(rewards) 
        # Compute the discounted returns at each timestep,
        # as 
        #      the sum of the gamma-discounted return at time t (G_t) + the reward at time t
        #
        # In O(N) time, where N is the number of time steps
        # (this definition of the discounted return G_t follows the definition of this quantity 
        # shown at page 44 of Sutton&Barto 2017 2nd draft)
        # G_t = r_(t+1) + r_(t+2) + ...
        
        # Given this formulation, the returns at each timestep t can be computed 
        # by re-using the computed future returns G_(t+1) to compute the current return G_t
        # G_t = r_(t+1) + gamma*G_(t+1)
        # G_(t-1) = r_t + gamma* G_t
        # (this follows a dynamic programming approach, with which we memorize solutions in order 
        # to avoid computing them multiple times)
        
        # This is correct since the above is equivalent to (see also page 46 of Sutton&Barto 2017 2nd draft)
        # G_(t-1) = r_t + gamma*r_(t+1) + gamma*gamma*r_(t+2) + ...
        
        
        ## Given the above, we calculate the returns at timestep t as: 
        #               gamma[t] * return[t] + reward[t]
        #
        ## We compute this starting from the last timestep to the first, in order
        ## to employ the formula presented above and avoid redundant computations that would be needed 
        ## if we were to do it from first to last.
        
        ## Hence, the queue "returns" will hold the returns in chronological order, from t=0 to t=n_steps
        ## thanks to the appendleft() function which allows to append to the position 0 in constant time O(1)
        ## a normal python list would instead require O(N) to do this.
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft( gamma*disc_return_t + rewards[t]   )    
            
        ## standardization of the returns is employed to make training more stable
        eps = np.finfo(np.float32).eps.item()
        ## eps is the smallest representable float, which is 
        # added to the standard deviation of the returns to avoid numerical instabilities        
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)
        
        # Line 7:
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()
        
        # Line 8: PyTorch prefers gradient descent 
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
    save_transitions()
    return scores

In [63]:
pong_hyperparameters = {
    "h_size": 16,
    "n_training_episodes": 10,
    "n_evaluation_episodes": 10,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-2,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

In [64]:
# Create policy and place it to the device
pong_policy = Policy(pong_hyperparameters["state_space"], pong_hyperparameters["action_space"], pong_hyperparameters["h_size"]).to(device)
pong_optimizer = optim.Adam(pong_policy.parameters(), lr=pong_hyperparameters["lr"])

In [65]:
scores = reinforce(pong_policy,
                   pong_optimizer,
                   pong_hyperparameters["n_training_episodes"], 
                   pong_hyperparameters["max_t"],
                   pong_hyperparameters["gamma"], 
                   100)

(998, 3, 210, 160)
(998, 1, 210, 160)
(998,)
(998,)
(998, 3, 210, 160)
(998, 1, 210, 160)
(998,)
(998,)
(998, 3, 210, 160)
(998, 1, 210, 160)
(998,)
(998,)
(998, 3, 210, 160)
(998, 1, 210, 160)
(998,)
(998,)
(998, 3, 210, 160)
(998, 1, 210, 160)
(998,)
(998,)
(998, 3, 210, 160)
(998, 1, 210, 160)
(998,)
(998,)
(998, 3, 210, 160)
(998, 1, 210, 160)
(998,)
(998,)


In [ ]:
print(len(observations))

7795


In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
  """
  Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
  :param env: The evaluation environment
  :param n_eval_episodes: Number of episode to evaluate the agent
  :param policy: The Reinforce agent
  """
  episode_rewards = []
  for episode in range(n_eval_episodes):
    state = env.reset()[0]
    step = 0
    terminated, truncated = False,False
    total_rewards_ep = 0
    
    for step in range(max_steps):
      action, _ = policy.act(state)
      new_state, reward, terminated, truncated, info = env.step(action)
      total_rewards_ep += reward
        
      if terminated or truncated:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

In [ ]:
evaluate_agent(eval_env, 
               pong_hyperparameters["max_t"], 
               pong_hyperparameters["n_evaluation_episodes"],
               pong_policy)

(-21.0, 0.0)